In [2]:
##import libraries needed
import pandas as pd
import os as os
import numpy as np
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import display, HTML

In [3]:
sql='''
select distinct bnf.presentation, presc.bnf_code, dose_per_unit, new_ome_multiplier, ome_dose
from richard.opioid_converter as opioid
inner join 
`ebmdatalab.richard.all_bnf_rx` as presc
ON CONCAT(
    SUBSTR(presc.bnf_code,0,9),
    'AA',
    SUBSTR(presc.bnf_code,-2,2)
  ) = CONCAT(
    SUBSTR(opioid.bnf_code,0,11),
    SUBSTR(opioid.bnf_code,-2,2)
  )
inner join 
hscic.bnf as bnf
ON
presc.bnf_code = bnf.presentation_code
'''
df_opioid_old = bq.cached_read(sql, csv_path=os.path.join('..','data','df_opioid_old_test.csv'))

In [4]:
df_opioid_old.head()

,presentation,bnf_code,dose_per_unit,new_ome_multiplier,ome_dose
0,Mezolar Matrix 25micrograms/hour transdermal p...,0407020A0BFABAE,1.8,100.0,180.0
1,Morphine sulfate 20mg/ml oral solution sugar free,0407020Q0AAA6A6,20.0,1.0,20.0
2,Physeptone 50mg/5ml solution for injection amp...,0407020M0BBAEAN,50.0,3.0,150.0
3,Solpadol 30mg/500mg capsules,0407010F0BGACAD,30.0,0.1,3.0
4,Morphine tartrate 15mg/1ml / Cyclizine tartrat...,040702020AAABAB,15.0,2.0,30.0


In [5]:
sql='''
WITH 
#vmp_amp AS 
#(SELECT DISTINCT id,
#                 nm,
#                 bnf_code,
#                 udfs
#FROM dmd.vmp #vmp table
#WHERE bnf_code IS NOT NULL
#UNION DISTINCT
#SELECT DISTINCT vmp,
#                nm,
#                bnf_code,
#                udfs
#FROM dmd.amp #amp table
#WHERE bnf_code IS NOT NULL)

#,
ing_level_rx AS
(select distinct ##distinct needed due to case statement for simple formulation
rx.bnf_code,
vmp.nm as name,
CASE
    WHEN vpi.strnt_nmrtr_uom = 258685003 THEN vpi.strnt_nmrtr_val / 1000 #creates miligram value from mcg value (uom = 258685003)
    WHEN vpi.strnt_nmrtr_uom = 258682000 THEN vpi.strnt_nmrtr_val * 1000 #creates miligram value from gram value (uom = 258682000)
    WHEN vpi.strnt_nmrtr_uom = 258684004 THEN vpi.strnt_nmrtr_val #normal miligram value (uom = 258684004)
    ELSE null
    END AS strnt_nmrtr_val_mg, #all listed drugs now in miligram rather than g or mcg
CASE
    WHEN vpi.strnt_dnmtr_uom = 258770004 THEN vpi.strnt_dnmtr_val * 1000 #some denominators listed as litre, so create mililitre value (uom = 258770004)
    WHEN ing.id=373492002 AND form.descr LIKE '%transdermal%' THEN COALESCE(vpi.strnt_dnmtr_val/72, 1/72) # creates 72 hour dose for fentanyl transdermal patches, as doses are per hour on DM+D)
    WHEN ing.id=387173000 AND form.descr LIKE '%transdermal%' AND strnt_nmrtr_val IN (5, 10, 15, 20) THEN COALESCE(vpi.strnt_dnmtr_val/168, 1/168) # creates 168 hour (7 day) dose for low-dose buprenorphine patch
    WHEN ing.id=387173000 AND form.descr LIKE '%transdermal%' AND strnt_nmrtr_val IN (35, 52.5, 70) THEN COALESCE(vpi.strnt_dnmtr_val/96, 1/96) # creates 96 hour dose for higher-dose buprenorphine patch
    ELSE vpi.strnt_dnmtr_val
    END AS strnt_dnmtr_val_ml, #denominator now in ml
ing.id as ing, 
ing.nm as ing_name,
COALESCE(udfs, 1) AS unit_dose,
CASE 
    WHEN form.descr LIKE '%injection%' THEN 'injection' #creates "injection" as route, regardless of whether injection or infusion. this also removes injection routes, e.g.
    WHEN descr LIKE '%infusion%' THEN 'injection'  #s/c, i/v etc, AS often injections have many licensed routes, which would multiply the row
        ELSE SUBSTR(form.descr, STRPOS(form.descr,".")+1) #takes the dosage form out of the string (e.g. tablet.oral) TO leave route.
    END AS simple_form 
from `ebmdatalab.richard.all_bnf_rx` as rx
INNER JOIN
dmd.vmp as vmp
ON CONCAT(SUBSTR(rx.bnf_code,0,9),'AA',SUBSTR(rx.bnf_code,-2,2)) = CONCAT(SUBSTR(vmp.bnf_code,0,11),SUBSTR(vmp.bnf_code,-2,2) )
INNER JOIN 
dmd.vpi as vpi
ON
vmp.id = vpi.vmp
INNER JOIN 
dmd.ing as ing
on
vpi.ing = ing.id
INNER JOIN
dmd.ont as ont
ON
vmp.id = ont.vmp
INNER JOIN 
dmd.ontformroute as form
ON
ont.form = form.cd
)

select 
#month, 
name, 
bnf_code,
unit_dose,
round(strnt_nmrtr_val_mg/coalesce(strnt_dnmtr_val_ml,1),3) as dose_per_unit,
round(SUM(ome*strnt_nmrtr_val_mg/coalesce(strnt_dnmtr_val_ml, 1)),2) as total_ome#all other products have usual dose - coalesce as solid dose forms do not have a denominator),2) as total_ome
from ing_level_rx as ing_rx
inner join richard.opioid_class as opioid
on
ing_rx.ing = opioid.id and ing_rx.simple_form = opioid.form
group by 
#month, 
name, bnf_code, ome, dose_per_unit, unit_dose
'''
df_opioid_new = bq.cached_read(sql, csv_path=os.path.join('..','data','df_opioid_new_test.csv'))

In [6]:
df_opioid_new.head(10)

,name,bnf_code,unit_dose,dose_per_unit,total_ome
0,Fentanyl 25micrograms/hour transdermal patches,0407020A0BFABAE,1,1.8,180.0
1,Morphine sulfate 20mg/ml oral solution sugar free,0407020Q0AAA6A6,1,20.0,20.0
2,Methadone 50mg/5ml solution for injection ampo...,0407020M0BBAEAN,5,10.0,30.0
3,Co-codamol 30mg/500mg capsules,0407010F0BGACAD,1,30.0,3.0
4,Morphine tartrate 15mg/1ml / Cyclizine tartrat...,040702020AAABAB,1,15.0,30.0
5,Pethidine 100mg/2ml solution for injection amp...,0407020V0AAAEAE,2,50.0,12.0
6,Tramadol 200mg modified-release capsules,040702040BJADAJ,1,200.0,20.0
7,Tramadol 150mg modified-release capsules,040702040BDADAI,1,150.0,15.0
8,Methadone 35mg/3.5ml solution for injection am...,0407020M0BBADAT,3.5,10.0,30.0
9,Morphine sulfate 10mg/2ml solution for injecti...,0407020Q0AABCBC,2,5.0,10.0


In [7]:
df_opioid_old.head(20)

,presentation,bnf_code,dose_per_unit,new_ome_multiplier,ome_dose
0,Mezolar Matrix 25micrograms/hour transdermal p...,0407020A0BFABAE,1.80,100.00,180.0
1,Morphine sulfate 20mg/ml oral solution sugar free,0407020Q0AAA6A6,20.00,1.00,20.0
2,Physeptone 50mg/5ml solution for injection amp...,0407020M0BBAEAN,50.00,3.00,150.0
3,Solpadol 30mg/500mg capsules,0407010F0BGACAD,30.00,0.10,3.0
4,Morphine tartrate 15mg/1ml / Cyclizine tartrat...,040702020AAABAB,15.00,2.00,30.0
5,Pethidine 100mg/2ml solution for injection amp...,0407020V0AAAEAE,100.00,0.24,24.0
6,Tramquel SR 200mg capsules,040702040BJADAJ,200.00,0.10,20.0
7,Zamadol SR 150mg capsules,040702040BDADAI,150.00,0.10,15.0
8,Physeptone 35mg/3.5ml solution for injection a...,0407020M0BBADAT,35.00,3.00,105.0
9,Morphine sulfate 10mg/2ml solution for injecti...,0407020Q0AABCBC,10.00,2.00,20.0


In [8]:
#df_opioid_new['dose_per_unit','total_ome'] = pd.to_numeric(df_opioid_new['dose_per_unit','total_ome'])
#df_opioid_old["month"] = pd.to_datetime(df_opioid_old["month"]) 
#df_opioid_new["month"] = pd.to_datetime(df_opioid_new["month"])
#df_opioid_new[['quantity','dose_per_unit','total_ome']] = df_opioid_new[['quantity','dose_per_unit','total_ome']].apply(pd.to_numeric, axis = 1)
#df_opioid_old[['quantity','dose_per_unit','total_ome']] = df_opioid_old[['quantity','dose_per_unit','total_ome']].apply(pd.to_numeric, axis = 1)

In [9]:
pd.concat([df_opioid_old,df_opioid_new]).drop_duplicates(keep=False)

,presentation,bnf_code,dose_per_unit,new_ome_multiplier,ome_dose,name,unit_dose,total_ome
0,Mezolar Matrix 25micrograms/hour transdermal p...,0407020A0BFABAE,1.800,100.0,180.0,NaN,NaN,NaN
1,Morphine sulfate 20mg/ml oral solution sugar free,0407020Q0AAA6A6,20.000,1.0,20.0,NaN,NaN,NaN
2,Physeptone 50mg/5ml solution for injection amp...,0407020M0BBAEAN,50.000,3.0,150.0,NaN,NaN,NaN
3,Solpadol 30mg/500mg capsules,0407010F0BGACAD,30.000,0.1,3.0,NaN,NaN,NaN
4,Morphine tartrate 15mg/1ml / Cyclizine tartrat...,040702020AAABAB,15.000,2.0,30.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
908,NaN,0407020A0BFAFBW,2.700,NaN,NaN,Fentanyl 37.5microgram/hour transdermal patches,1,270.00
909,NaN,0407020A0BRAKAU,0.864,NaN,NaN,Fentanyl 12micrograms/hour transdermal patches,1,86.40
910,NaN,040702040BUABAD,150.000,NaN,NaN,Tramadol 150mg modified-release tablets,1,15.00
911,NaN,0407020A0BTAABX,0.133,NaN,NaN,Fentanyl 133microgram sublingual tablets sugar...,1,17.29


In [11]:
new_df = pd.merge(df_opioid_old,df_opioid_new,  how='outer', on=['bnf_code'])

In [13]:
new_df['ome_diff']=round(new_df['total_ome']-new_df['ome_dose'],1)

In [15]:
new_df['ome_diff_multi']=new_df['total_ome']/new_df['ome_dose']

In [17]:
new_df.to_csv('differences.csv')

In [124]:
diff_df = new_df[new_df.ome_diff != 0]

In [125]:
diff_df.head()

,bnf_name,bnf_code,dose_per_unit_x,quantity_x,total_ome_x,dose_per_unit_y,quantity_y,total_ome_y,ome_diff,ome_diff_multi
4,Actiq_Loz 400mcg,0407020A0BCABAJ,0.4,57588.0,2.303520e+06,0.4,57588.0,2.994576e+06,-691056.0,0.769231
5,Papaveretum_Inj 40mg/ml 1ml Amp,0407020ABAAAKAK,40.0,529.0,2.116000e+04,40.0,529.0,1.629320e+05,-141772.0,0.129870
9,Shortec 10mg capsules,0407020ADBJABAD,10.0,27341088.0,5.468218e+08,10.0,26851812.0,4.027772e+08,144044580.0,1.357628
10,Zomorph 100mg modified-release capsules,0407020Q0BKADEB,100.0,15281912.0,1.528191e+09,100.0,15143053.0,1.514305e+09,13885900.0,1.009170
11,Mezolar Matrix 37.5microgram/hour transdermal ...,0407020A0BFAFBW,0.3,498340.0,1.495020e+07,2.7,486068.0,1.312384e+08,-116288159.0,0.113916


In [57]:
diff_df_

,month,bnf_name,bnf_code,dose_per_unit_x,quantity_x,total_ome_x,dose_per_unit_y,quantity_y,total_ome_y,ome_diff
7,2013-04-01 00:00:00+00:00,Fentanyl_Loz 1.2mg,0407020A0AAANAN,1.2,2751.0,330120.0,31.20,2751.0,429156.0,-99036.0
10,2013-09-01 00:00:00+00:00,Diamorph HCl_(S),0407020K0AABQBQ,NaN,60.0,NaN,NaN,NaN,NaN,NaN
11,2012-04-01 00:00:00+00:00,Methadone HCl_(S),0407020M0AAAFAF,NaN,4.0,NaN,NaN,NaN,NaN,NaN
13,2015-06-01 00:00:00+00:00,Fentanyl 50micrograms/dose nasal spray,0407020A0AABHBH,0.5,11.0,880.0,0.25,11.0,88.0,792.0
18,2018-05-01 00:00:00+00:00,Actiq 1.2mg lozenges with integral oromucosal ...,0407020A0BCAEAN,1.2,595.0,71400.0,4.80,595.0,92820.0,-21420.0
...,...,...,...,...,...,...,...,...,...,...
62221,2017-12-01 00:00:00+00:00,Morphine sulfate 10mg/5ml soln unit dose vials...,0407020Q0AAGBGB,NaN,NaN,NaN,2.00,20.0,40.0,NaN
62222,2017-12-01 00:00:00+00:00,Papaveretum 30mg/1ml solution for injection am...,0407020ABAAAPAP,NaN,NaN,NaN,30.00,4.0,924.0,NaN
62223,2010-09-01 00:00:00+00:00,Propain Plus_Capl,0407010X0BKACA0,NaN,NaN,NaN,90.00,836.0,836.0,NaN
62224,2010-09-01 00:00:00+00:00,Solpadeine Plus_Tab Solb,0407010X0BMABA0,NaN,NaN,NaN,904.00,15650.0,12520.0,NaN


In [137]:
agg_df = diff_df.groupby(['bnf_name', 'bnf_code', 'ome_diff_multi'])['ome_diff'].agg('sum')

In [138]:
agg_df=agg_df.to_frame()

In [135]:
agg_df.sort_values(by='ome_diff_multi', ascending=False)

,,,ome_diff
bnf_name,bnf_code,ome_diff_multi,
Morph Sulph_Inj 1mg/ml 10ml Pfs,0407020Q0AAFJFJ,300.000000,2076256.0
Minjet_Morph Sulph 1mg/ml 10ml Pfs,0407020Q0BEABFJ,300.000000,29302.0
Morphine sulfate 10mg/10ml inj pre-filled syringes,0407020Q0AAFJFJ,300.000000,244582.0
Morphine sulfate 10mg/10ml inj Minijet pre-filled syringes,0407020Q0BEABFJ,300.000000,29900.0
Morph Sulf_Inj 1mg/ml 10ml Pfs,0407020Q0AAFJFJ,300.000000,289432.0
...,...,...,...
Papaveret/Hyoscine_Inj 15.4/0.4mg 1ml Am,0407020ABAAAIAI,0.129870,-42510.2
Papaveretum_Inj 40mg/ml 1ml Amp,0407020ABAAAKAK,0.129870,-141772.0
Papaveretum 40mg/1ml solution for injection ampoules,0407020ABAAAKAK,0.129870,-15544.0


In [129]:
agg_df.reset_index()

,bnf_name,bnf_code,ome_diff_multi,ome_diff
0,Abstral 100microgram sublingual tablets,0407020A0BJAAAW,1.018398,156871.0
1,Abstral 200microgram sublingual tablets,0407020A0BJABAX,1.007851,102258.0
2,Abstral 300microgram sublingual tablets,0407020A0BJACAY,1.012531,63648.0
3,Abstral 400microgram sublingual tablets,0407020A0BJADAZ,1.011103,174564.0
4,Abstral 600microgram sublingual tablets,0407020A0BJAEBA,1.013586,104442.0
...,...,...,...,...
764,Zydol XL 200mg tablets,040702040BBAHAE,1.010032,34640.0
765,Zydol XL 300mg tablets,040702040BBAIAM,1.016408,35640.0
766,Zydol XL 400mg tablets,040702040BBAJAN,1.011008,43200.0
767,Zydol_Inj 100mg/2ml Amp,040702040BBABAB,2.000000,260.0


In [131]:
#agg_df = agg_df[agg_df.ome_diff <1 and >-1]
agg_df = agg_df[(agg_df['ome_diff']<1) & (agg_df['ome_diff']<-1)]

In [139]:
display(HTML(agg_df.sort_values(by='ome_diff_multi', ascending=False).to_html()))

,,,ome_diff
bnf_name,bnf_code,ome_diff_multi,
Morph Sulph_Inj 1mg/ml 10ml Pfs,0407020Q0AAFJFJ,300.000000,2076256.0
Minjet_Morph Sulph 1mg/ml 10ml Pfs,0407020Q0BEABFJ,300.000000,29302.0
Morphine sulfate 10mg/10ml inj pre-filled syringes,0407020Q0AAFJFJ,300.000000,244582.0
Morphine sulfate 10mg/10ml inj Minijet pre-filled syringes,0407020Q0BEABFJ,300.000000,29900.0
Morph Sulf_Inj 1mg/ml 10ml Pfs,0407020Q0AAFJFJ,300.000000,289432.0
Minjet_Morph Sulf 1mg/ml 10ml Pfs,0407020Q0BEABFJ,300.000000,2990.0
Morphine sulfate 50mg/50ml solution for infusion vials,0407020Q0AAENEN,50.286369,172108.0
Morphine sulfate 100mg/50ml solution for infusion vials,0407020Q0AAELEL,50.000000,42336.0
Morph Sulf_Inj 100mg/50ml Vl,0407020Q0AAELEL,50.000000,5488.0
